In [1]:
import os
import json
import joblib
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
import pickle


In [2]:


# %% [markdown]
# # 5. Integration Testing (Neural + Symbolic)
# This notebook focuses on:
# 1. Loading the trained neural forecasting model.
# 2. Instantiating the symbolic countermeasure suggester.
# 3. Simulating a workflow:
#     a. Get a threat forecast from the neural model (using mock input).
#     b. Pass the forecast to the symbolic suggester.
#     c. Display the combined output.
# 4. Discussing the benefits of this integrated approach.

# %%


# %% [markdown]
# ## Setup Paths and Load Components

# %%
BASE_DIR = os.path.dirname(os.getcwd())
MODEL_DIR_NEURAL = os.path.join(BASE_DIR, 'models', 'neural_forecasting')
MODEL_DIR_GENERAL = os.path.join(BASE_DIR, 'models')  # For preprocessor

# Find the latest .pt model (for PyTorch)
neural_model_files = [f for f in os.listdir(MODEL_DIR_NEURAL) if f.endswith('.pt')]
if not neural_model_files:
    raise FileNotFoundError(f"No trained neural model (.pt) found in {MODEL_DIR_NEURAL}")

LATEST_MODEL_NAME = sorted(neural_model_files, reverse=True)[0]
neural_model_path = os.path.join(MODEL_DIR_NEURAL, LATEST_MODEL_NAME)
#preprocessor_path = os.path.join(MODEL_DIR_GENERAL, 'preprocessor.joblib')#replace
PROCESSED_DIR = os.path.join(BASE_DIR, 'data', 'processed')
preprocessor_path = os.path.join(PROCESSED_DIR, 'tfidf_vectorizer.pkl')
with open(preprocessor_path, 'rb') as f:
    preprocessor = pickle.load(f)

print(f"Using Neural Model: {neural_model_path}")
print(f"Using Preprocessor: {preprocessor_path}")

# Try loading model and preprocessor
try:
    class MLPModel(nn.Module):
        def __init__(self, input_dim):
            super(MLPModel, self).__init__()
            self.model = nn.Sequential(
                nn.Linear(input_dim, 128),
                nn.ReLU(),
                nn.Dropout(0.3),
                nn.Linear(128, 64),
                nn.ReLU(),
                nn.Dropout(0.3),
                nn.Linear(64, 32),
                nn.ReLU(),
                nn.Linear(32, 1),
                nn.Sigmoid()
            )

        def forward(self, x):
            return self.model(x)

    input_dim = joblib.load(os.path.join(MODEL_DIR_GENERAL, 'input_dim.pkl'))
    neural_forecaster = MLPModel(input_dim)
    neural_forecaster.load_state_dict(torch.load(neural_model_path, map_location=torch.device('cpu')))
    neural_forecaster.eval()

    #preprocessor = joblib.load(preprocessor_path) #replace

except Exception as e:
    print(f"Error loading model or preprocessor: {e}")
    print("Using dummy model and preprocessor for fallback.")

    class DummyModel:
        def __call__(self, x): return torch.tensor([[0.8]])

    class DummyPreprocessor:
        def transform(self, x): return x
        def get_feature_names_out(self, input_features=None): return input_features or ['feature1', 'feature2']

    if 'neural_forecaster' not in locals():
        neural_forecaster = DummyModel()
    if 'preprocessor' not in locals():
        preprocessor = DummyPreprocessor()

print("\nLoaded Model Architecture:\n", neural_forecaster)

# %% [markdown]
# ## Add Symbolic Reasoning (Knowledge Base)

# %%
KNOWLEDGE_BASE = {
    "threat_types_details": {
        "IED_urban": {"urgency": "high", "potential_impact": "severe", "indicators": ["suspicious_package", "chatter_increase"]},
        "armed_assault_gov": {"urgency": "critical", "potential_impact": "severe", "indicators": ["weapon_sighting", "reconnaissance_activity"]},
        "cyber_attack_infra": {"urgency": "high", "potential_impact": "high", "indicators": ["phishing_attempts", "network_anomaly"]},
        "propaganda_online": {"urgency": "medium", "potential_impact": "moderate", "indicators": ["fake_news_spike", "hate_speech_increase"]}
    },
    "countermeasures": {
        "cm_001": {"name": "Increased Patrols (Visible Presence)", "resources_needed": ["personnel_low"], "mitigates_threats": ["IED_urban", "armed_assault_gov"], "effectiveness_score": 0.6, "ethical_impact": "low", "legality_check": "standard_procedure"},
        "cm_002": {"name": "EOD Sweeps & Checkpoints", "resources_needed": ["personnel_medium", "eod_team", "equipment_basic"], "mitigates_threats": ["IED_urban"], "effectiveness_score": 0.85, "ethical_impact": "medium_low", "legality_check": "requires_justification"},
        "cm_003": {"name": "Enhanced Cybersecurity Monitoring", "resources_needed": ["cyber_team_skilled", "monitoring_tools_advanced"], "mitigates_threats": ["cyber_attack_infra"], "effectiveness_score": 0.75, "ethical_impact": "low", "legality_check": "privacy_compliant"},
        "cm_004": {"name": "Public Awareness Campaign (Disinformation)", "resources_needed": ["media_team", "communication_channels"], "mitigates_threats": ["propaganda_online", "IED_urban"], "effectiveness_score": 0.5, "ethical_impact": "very_low", "legality_check": "standard_procedure"},
        "cm_005": {"name": "Targeted Intelligence Operation", "resources_needed": ["intel_team_specialized", "surveillance_auth"], "mitigates_threats": ["armed_assault_gov", "IED_urban"], "effectiveness_score": 0.9, "ethical_impact": "high", "legality_check": "strict_oversight_warrant_required"},
        "cm_006": {"name": "De-escalation & Community Engagement", "resources_needed": ["community_liaisons", "negotiators"], "mitigates_threats": ["propaganda_online"], "effectiveness_score": 0.65, "ethical_impact": "positive", "legality_check": "standard_procedure"}
    },
    "rules_of_engagement": [
        {"id": "ROE_01", "condition": "IF threat_urgency == 'critical' AND potential_impact == 'severe'", "action": "Prioritize countermeasures with high effectiveness, consider higher ethical impact if necessary.", "priority_modifier": 1.5},
        {"id": "ROE_02", "condition": "IF threat_type == 'propaganda_online'", "action": "Favor non-intrusive countermeasures like awareness and engagement.", "cm_preference": ["cm_004", "cm_006"]},
        {"id": "ROE_03", "condition": "IF predicted_prob < 0.6", "action": "Suggest low-intensity monitoring and preparedness actions first.", "initial_cm_filter": "low_resource_low_ethical_impact"}
    ]
}

class SymbolicCountermeasureSuggester:
    def __init__(self, kb):
        self.kb = kb

    def get_threat_details(self, threat_type_id):
        return self.kb["threat_types_details"].get(threat_type_id)

    def suggest_countermeasures(self, threat_forecast):
        suggestions = []
        threat_type_id = threat_forecast.get("threat_type_id")
        if not threat_type_id:
            return {"error": "Threat type ID missing."}

        threat_details = self.get_threat_details(threat_type_id)
        if not threat_details:
            return {"error": f"Unknown threat type: {threat_type_id}"}

        applicable_cms = []
        for cm_id, cm_data in self.kb["countermeasures"].items():
            if threat_type_id in cm_data.get("mitigates_threats", []):
                cm_score = cm_data.get("effectiveness_score", 0.5)
                priority_mod = 1.0
                if threat_details.get("urgency") == "critical" and threat_details.get("potential_impact") == "severe":
                    priority_mod *= self.kb["rules_of_engagement"][0].get("priority_modifier", 1.0)

                prob = threat_forecast.get("predicted_prob_high_casualty", 0.5)
                cm_score *= (prob + 0.5) * priority_mod

                if prob < 0.6 and self.kb["rules_of_engagement"][2].get("initial_cm_filter") == "low_resource_low_ethical_impact":
                    if cm_data.get("ethical_impact") not in ["low", "very_low"]:
                        continue
                    if not any("low" in r for r in cm_data.get("resources_needed", [])):
                        continue

                applicable_cms.append({
                    "id": cm_id,
                    "name": cm_data["name"],
                    "base_effectiveness": cm_data.get("effectiveness_score", 0.5),
                    "calculated_score": round(cm_score, 3),
                    "ethical_impact": cm_data.get("ethical_impact"),
                    "legality_check": cm_data.get("legality_check"),
                    "resources": cm_data.get("resources_needed")
                })

        applicable_cms.sort(key=lambda x: x["calculated_score"], reverse=True)
        explanation = f"Countermeasures for '{threat_type_id}' (Predicted Prob High Casualty: {threat_forecast.get('predicted_prob_high_casualty', 'N/A'):.2f}). "
        explanation += f"Threat urgency (from KB): {threat_details.get('urgency', 'N/A')}, Potential Impact (from KB): {threat_details.get('potential_impact', 'N/A')}. "
        if threat_forecast.get("predicted_prob_high_casualty", 1.0) < 0.6:
            explanation += "Low probability forecast, favoring precautionary measures. "

        return {"suggestions": applicable_cms, "explanation": explanation.strip()}

symbolic_suggester = SymbolicCountermeasureSuggester(KNOWLEDGE_BASE)
print("Symbolic suggester instantiated.")


Using Neural Model: c:\Users\rahul\Desktop\Deeplcapstone\models\neural_forecasting\mlp_threat_forecaster.pt
Using Preprocessor: c:\Users\rahul\Desktop\Deeplcapstone\data\processed\tfidf_vectorizer.pkl

Loaded Model Architecture:
 MLPModel(
  (model): Sequential(
    (0): Linear(in_features=5000, out_features=128, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.3, inplace=False)
    (3): Linear(in_features=128, out_features=64, bias=True)
    (4): ReLU()
    (5): Dropout(p=0.3, inplace=False)
    (6): Linear(in_features=64, out_features=32, bias=True)
    (7): ReLU()
    (8): Linear(in_features=32, out_features=1, bias=True)
    (9): Sigmoid()
  )
)
Symbolic suggester instantiated.
